# Example 0.1: plain erddappy

Copied from the [errdappy intro](https://ioos.github.io/erddapy/quick_intro-output.html#Longer-introduction)

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap",
    response="csv",
)
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

e.constraints = {
    "time>=": "2016-07-10T00:00:00Z",
    "time<=": "2017-02-10T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}


In [ ]:
df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
)
df.head()

# Example 0.2: class wrapping erddapy
Emulating the UI of [argopy](https://github.com/euroargodev/argopy)

In [ ]:
from gliderpy.fetchers import GliderDataFetcher


glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
df = glider_grab.to_pandas()
df.head()

Try with a different datset

In [ ]:
glider_grab = GliderDataFetcher()
glider_grab.fetcher.dataset_id = "sp022-20170209T1616"

df = glider_grab.to_pandas()
df.head()

# Example 0.3: implement time and space constraints
we can make a direct call to erddapy's `constraints`, this isn't very elegant though

In [ ]:
glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"

glider_grab.fetcher.constraints = {
    "time>=": "2016-07-10T00:00:00Z",
    "time<=": "2016-09-15T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}

df = glider_grab.to_pandas()
df.tail()

Let's try making a method of the class to do this more neatly

In [ ]:
glider_grab = GliderDataFetcher()

# In this case we don't want to specify a dataset_id, we want all gliders in that query.
glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
min_lat, max_lat, min_lon, max_lon = 10, 40.5, -71, 180
start_time, end_time = "2016-08-1","2016-09-18"
glider_grab.query(
    min_lat,
    max_lat,
    min_lon,
    max_lon,
    start_time,
    end_time
)

df = glider_grab.to_pandas()
df.tail()

In [ ]:
df.describe()

# Example 0.4 find glider datasets on server

Calling get_ids from the class searches for all databases tagged as glider

In [ ]:
from gliderpy.fetchers import DatasetList

datasets = DatasetList()
datasets.get_ids()

Let's look for the ones from UW

In [ ]:
datasets = DatasetList()
datasets.search_terms = ["UW"]
uw_glider_ids = datasets.get_ids()
uw_glider_ids

We'll pass that id back into our original data fetcher

In [ ]:
uw_grab = GliderDataFetcher()
uw_grab.fetcher.dataset_id = uw_glider_ids[18]

df = uw_grab.to_pandas(
)
df.head()

## Example 0.5 Now let's plot the track map for a dataset

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


def plot_track(df):
    x = df["longitude (degrees_east)"]
    y = df["latitude (degrees_north)"]
    dx, dy = 2, 4

    fig, ax = plt.subplots(
        figsize=(9, 9), subplot_kw={"projection": ccrs.PlateCarree()}
    )
    cs = ax.scatter(
        x, y, c=df["temperature (Celsius)"], s=50, alpha=0.5, edgecolor="none"
    )
    cbar = fig.colorbar(
        cs, orientation="vertical", fraction=0.1, shrink=0.9, extend="both"
    )
    ax.coastlines("10m")
    ax.set_extent([x.min() - dx, x.max() + dx, y.min() - dy, y.max() + dy])
    return fig, ax

In [ ]:
fig, ax = plot_track(df)

## Now the transect for a given variable in that dataset

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import palettable
from palettable.cmocean.sequential import Haline_20

cmap = Haline_20.mpl_colormap


def plot_transect(df,var):
    fig, ax = plt.subplots(figsize=(17, 2))
    cs = ax.scatter(
        df.index,
        df["depth (m)"],
        s=15,
        c=df[var],
        marker="o",
        edgecolor="none",
        cmap=cmap,
    )

    ax.invert_yaxis()
    xfmt = mdates.DateFormatter("%H:%Mh\n%d-%b")
    ax.xaxis.set_major_formatter(xfmt)

    cbar = fig.colorbar(cs, orientation="vertical", extend="both")
    cbar.ax.set_ylabel(var)
    ax.set_ylabel("Depth (m)")
    return fig, ax

In [ ]:
fig, ax = plot_transect(df, var="salinity (1)")